In [1]:
import pandas as pd
import re

In [4]:
# Creating the dataframe

df = pd.read_csv('./chordonomicon.csv') # Shortened version of real chordonomicon

df.head()

/var/folders/cf/lppdt0wd6b937186z5byb7c00000gn/T/ipykernel_72579/2285757688.py:3: DtypeWarning: Columns (3,4,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./chordonomicon.csv') # Shortened version of real chordonomicon


,id,chords,s_artist,release_date,genres,decade,main_genre,rock_genre,parts
0,1,<intro_1> C <verse_1> F C E7 Amin C F C G7 C F...,artist_0,1977,classic country pop,1970.0,pop,NaN,yes
1,2,<intro_1> E D A/Cs E D A/Cs <verse_1> E D A/Cs...,artist_1,2003-01-01,"alternative metal""alternative rock""nu metal""pe...",2000.0,metal,pop rock,yes
2,3,<intro_1> D Dmaj7 D Dmaj7 <verse_1> Emin A D G...,artist_2,2022-09-23,NaN,2020.0,NaN,NaN,yes
3,4,<intro_1> C <verse_1> G C G C <chorus_1> F Dmi...,artist_3,2023-02-10,modern country pop,2020.0,pop,NaN,yes
4,5,<intro_1> C G C G <verse_1> C G C G C Bmin Emi...,artist_4,2018-08-24,"classic opm""opm",2010.0,NaN,NaN,yes


In [5]:
df['chords'] = df['chords'].apply(lambda s:
    ' '.join([re.sub(r"/[^/]*$", "", chord) # Remove inversions
              for chord in s.split()]
))

all_songs = df['chords'].str.replace(r'<.*?>', '', regex=True).str.strip() # Remove verse notations

In [6]:

def root_checker(chord):
    if len(chord) == 1:
        return chord[0]

    if chord[1] == 'b' or (chord[1] == 's' and (len(chord) < 3 or chord[2] != 'u')): # Check for flat, sharp, and sus
        return chord[:2]
    
    return chord[0]  # Returns chord

all_modifiers = [
    '',          # major triad
    'min',       # minor triad
    '7',         # dominant seventh
    'maj7',      # major seventh
    'min7',      # minor seventh
    'dim',       # diminished triad
    'sus4',      # suspended fourth
    '6',         # added 6th
    '9',         # dominant 9
    'min9',      # minor 9
    'maj9',      # major 9
    '13',        # dominant 13
]
all_notes = ["Ab", "A", "As", "Bb", "B", "C", "Cs", "Db", "D", "Ds", "Eb", "E", "F", "Fs", "Gb", "G", "Gs"]
note_conversion = {
    # Sharps
    'As': 'A#', 'Bs': 'C', 'Cs': 'C#', 'Ds': 'D#', 'Es': 'F', 
    'Fs': 'F#', 'Gs': 'G#',
    # Flats
    'Bb': 'A#', 'Cb': 'B', 'Db': 'C#', 'Eb': 'D#', 'Fb': 'E',
    'Gb': 'F#', 'Ab': 'G#'
}

valid_songs = []


for song in all_songs:
    split_song = song.split()
    valid_song = True
    modified_chords = []  # Store modified chords

    for chord in split_song:
        root = root_checker(chord)
        rootless_chord = chord[len(root):]
        
            
        if rootless_chord == '' or rootless_chord is None:
                rootless_chord = 'maj'  # Add default modifier
            
            
        modified_chord = note_conversion.get(root, root) + rootless_chord
        modified_chords.append(modified_chord)  # Store modified version
        
        if rootless_chord not in all_modifiers:
            valid_song = False
            break

    if valid_song:
        valid_songs.append(' '.join(modified_chords))
        


NameError: name 'rootless_chord' is not defined

In [ ]:
cutonomicon = pd.DataFrame(valid_songs, columns=['chords'])

cutonomicon.to_csv('./cutonomicon')
print(cutonomicon)

                                                  chords
0      C  F C E7 Amin C F C G7 C F C E7 Amin C F G7 C...
1      E D A E D A  E D A E D A E D A E D A C  E G D ...
2      D Dmaj7 D Dmaj7  Emin A D G Emin A D G Emin A ...
3      C  G C G C  F Dmin G Dmin G C  G C  F Dmin G D...
4      C G C G  C G C G C Bmin Emin Amin D  G C D G C...
...                                                  ...
18197  C F C F G C F C  F C F G C C7  F Fs G C C7 F G...
18198  Fs E  Csmin E Csmin A C E A C E  Csmin E Csmin...
18199  E A Bmin A D E A Bmin A D E A Bmin  A D E A Bm...
18200  Bmin G Bmin Fsmin Fsmin7  Bmin Fsmin Bmin Fsmi...
18201  Emin Amin Emin D Emin D Emin D C D C D Emin  D...

[18202 rows x 1 columns]
